<style>
h1{font-family: 'lucida handwriting'}
</style>
<center>

# **TP**<br> ontologie
</center>

### **CHERGUI Moad**  
IID3


In [99]:


import spacy
""""
import spacy.cli
spacy.cli.download("en_core_web_sm")
#"""

nlp = spacy.load("en_core_web_sm")

texte = "There are courses and laboratory courses. Homeworks are part of courses. Courses are organized by teachers. Teachers are either professors or assistants. Professors teach courses while assistants only teach laboratory courses."
doc = nlp(texte)

In [100]:
concepts = set()
relations = []

# Extraction des concepts (noms propres + noms communs importants)
for token in doc:
    if token.pos_ in ["PROPN", "NOUN"]:
        concepts.add(token.text)

In [101]:
relations = []
for token in doc:
    if token.pos_ in ['VERB', 'AUX']:
        subj = None
        obj = None
        prep = None
        for child in token.children:
            if child.dep_ in ['nsubj', 'nsubjpass']:
                subj = child.text
            elif child.dep_ in ['dobj', 'attr']:
                obj = child.text
            elif child.dep_ in ['prep', 'agent']:
                prep = child.text
                for gchild in child.children:
                    if gchild.dep_ == 'pobj':
                        obj = gchild.text
                        break
        if subj and obj:
            relation = token.lemma_
            if prep:
                relation += ' ' + prep
            relations.append((subj, relation, obj))

# Affichage
print("Concepts extracted from text:")
for c in sorted(concepts):
    print("-", c)

print("\nRelations extracted from text:")
for s, v, o in relations:
    print(f"{s} --{v}--> {o}")

Concepts extracted from text:
- Courses
- Homeworks
- Professors
- Teachers
- assistants
- courses
- laboratory
- part
- professors
- teachers

Relations extracted from text:
Homeworks --be--> part
Courses --organize by--> teachers
Teachers --be--> professors
Professors --teach--> courses
assistants --teach--> courses


In [102]:
from rdflib import Graph, Namespace, RDF, RDFS

# Création du graphe
g = Graph()
EX = Namespace("http://example.org/ontology/")

g.bind("ex", EX)

# ======================
# Classes based on extracted concepts
# ======================
g.add((EX.Course, RDF.type, RDFS.Class))
g.add((EX.Homework, RDF.type, RDFS.Class))
g.add((EX.Teacher, RDF.type, RDFS.Class))
g.add((EX.Professor, RDF.type, RDFS.Class))
g.add((EX.Assistant, RDF.type, RDFS.Class))
g.add((EX.LaboratoryCourse, RDF.type, RDFS.Class))

# Hierarchy based on text
g.add((EX.Professor, RDFS.subClassOf, EX.Teacher))
g.add((EX.Assistant, RDFS.subClassOf, EX.Teacher))
g.add((EX.LaboratoryCourse, RDFS.subClassOf, EX.Course))

# ======================
# Properties based on extracted relations
# ======================
g.add((EX.partOf, RDF.type, RDF.Property))
g.add((EX.organizedBy, RDF.type, RDF.Property))
g.add((EX.teach, RDF.type, RDF.Property))

# Domains and ranges
g.add((EX.partOf, RDFS.domain, EX.Homework))
g.add((EX.partOf, RDFS.range, EX.Course))

g.add((EX.organizedBy, RDFS.domain, EX.Course))
g.add((EX.organizedBy, RDFS.range, EX.Teacher))

g.add((EX.teach, RDFS.domain, EX.Teacher))
g.add((EX.teach, RDFS.range, EX.Course))



<Graph identifier=Nf770ab3d618d4a7b842d376250ae9093 (<class 'rdflib.graph.Graph'>)>

In [103]:
g.serialize("ontology.ttl", format="turtle")
print("Ontology file ontology.ttl created successfully")

Ontology file ontology.ttl created successfully
